## init

In [ ]:
import time

from icecream import ic
from loguru import logger as lg

from harmony_automation import core as ha

ha.login()

## load dat

In [ ]:
df = ha.return_data()
m = df.ticket_num.isna()
assert df[m].shape[0] == 0, "Missing ticket numbers in data"

## close logs

In [ ]:
# filter for only logs to close
m = df["log_action"].str.strip().str.lower() == "n"
logs_to_close = df[m].copy()
logs_to_close = [str(x) for x in logs_to_close.ticket_num.tolist()]
ic(len(logs_to_close))
completed_list = []

In [ ]:
for i in range(len(logs_to_close)):
    ticket_num = logs_to_close[i]

    ic(i)
    ic(ticket_num)
    remaining = len(logs_to_close) - (i + 1)
    ic(remaining)

    if ticket_num not in completed_list:
        ha.search_ticket_num(ticket_num)
        ha.view_log()
        ha.click_edit_log()
        ha.edit_res()
        ha.submit_res()
        time.sleep(0.5)
        completed_list.append(ticket_num)
        lg.info(f"closed {ticket_num}")

## re-assign

In [ ]:
# filter for only logs to re-assign
m = df["log_action"].str.strip().str.lower() != "n"
logs_to_reassign = df[m].copy()
logs_to_reassign = logs_to_reassign.reset_index(drop=True).copy()
ic(len(logs_to_reassign))
reassigned_list = []

In [ ]:
for i in range(len(logs_to_reassign)):
    ticket_num = str(logs_to_reassign.loc[i]["ticket_num"])
    note = str(logs_to_reassign.loc[i]["log_action"])

    ic(i)
    ic(ticket_num)
    remaining = len(logs_to_reassign) - (i + 1)
    ic(remaining)

    if ticket_num not in reassigned_list:
        ha.search_ticket_num(ticket_num)
        ha.reassign_log(note)
        time.sleep(1.8)
        reassigned_list.append(ticket_num)
        lg.info(f"re-assigned {ticket_num} w/ note: {note}")

In [ ]:
ha.driver.quit()